In [1]:
import os
import numpy as np
import cv2

In [2]:
y = np.load("train_labels.npy")
X = np.load("normalized_train_rgb_data.npy")

In [10]:
X.shape

(50000, 64, 64, 3)

In [3]:
y.shape

(59823, 10)

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available: 1


In [5]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

In [6]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
output = Dense(10, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = output)

#Freeze first layers
for layer in base_model.layers[:20]:
    layer.trainable = False
    
#Unfreeze the remaining layers to allow fine-tuning
for layer in base_model.layers[20:]:
    layer.trainable = True

In [7]:
from keras.optimizers import Adam
from keras import metrics
adam = Adam(learning_rate=0.0001)
model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy', metrics.Precision(), metrics.Recall()])

In [8]:
history = model.fit(X, y, epochs=10, batch_size=128, validation_split=0.1)

Epoch 1/10
352/352 [==============================] - 60s 126ms/step - loss: 1.9410 - accuracy: 0.3058 - precision: 0.4154 - recall: 0.0885 - val_loss: 4.0358 - val_accuracy: 8.0000e-04 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/10
352/352 [==============================] - 48s 135ms/step - loss: 1.3081 - accuracy: 0.5012 - precision: 0.6170 - recall: 0.3129 - val_loss: 8.4217 - val_accuracy: 0.0076 - val_precision: 0.0103 - val_recall: 0.0074
Epoch 3/10
352/352 [==============================] - 51s 144ms/step - loss: 1.1705 - accuracy: 0.5543 - precision: 0.6619 - recall: 0.3976 - val_loss: 8.9079 - val_accuracy: 0.0082 - val_precision: 0.0094 - val_recall: 0.0072
Epoch 4/10
352/352 [==============================] - 51s 144ms/step - loss: 1.1065 - accuracy: 0.5786 - precision: 0.6778 - recall: 0.4365 - val_loss: 9.0360 - val_accuracy: 0.0054 - val_precision: 0.0069 - val_recall: 0.0050
Epoch 5/10
352/352 [==============================] - 51s 144ms/step - loss: 1.0

In [9]:
loss, accuracy = model.evaluate(X, y)
print(f'Validation Accuracy: {accuracy}')

ValueError: Data cardinality is ambiguous:
  x sizes: 50000
  y sizes: 59823
Make sure all arrays contain the same number of samples.